# Regulation simulation

## Load GEMs

In [5]:
import cobra

In [2]:
import scipy

In [1]:
from cobra.io import load_matlab_model

In [42]:
mat_path = "../data/iJO1366.mat"
ecoli_model = load_matlab_model(mat_path)

This model seems to have metCharge instead of metCharges field. Will use metCharge for what metCharges represents.
No defined compartments in model iJO1366. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e, p


In [43]:
ecoli_model

Name,iJO1366
Memory address,0x07f087442ea60
Number of metabolites,1805
Number of reactions,2583
Number of groups,39
Objective expression,1.0*BIOMASS_Ec_iJO1366_core_53p95M - 1.0*BIOMASS_Ec_iJO1366_core_53p95M_reverse_5c8b1
Compartments,"c, e, p"


In [44]:
model = ecoli_model

## Change the upper and lower bound

In [6]:
cobra_config = cobra.Configuration()

In [7]:
cobra_config.bounds

(-1000.0, 1000.0)

In [36]:
cobra.Reaction("SUCCtex")

Reaction identifier,SUCCtex
Name,
Memory address,0x7f08906697f0
Stoichiometry,--> -->
GPR,
Lower bound,0.0
Upper bound,1000.0


In [37]:
SUCCtex = model.reactions.get_by_id("SUCCtex")
SUCCtex

Reaction identifier,SUCCtex
Name,Succinate transport via diffusion (extracellular to periplasm)
Memory address,0x7f0880c1f5e0
Stoichiometry,succ_e <-- succ_p Succinate <-- Succinate
GPR,b0241 or b0929 or b1377 or b2215
Lower bound,-400
Upper bound,-400


In [38]:
SUCCtex.bounds = (-400, -400) #didn't change the original bound of the model

In [39]:
SUCCtex

Reaction identifier,SUCCtex
Name,Succinate transport via diffusion (extracellular to periplasm)
Memory address,0x7f0880c1f5e0
Stoichiometry,succ_e <-- succ_p Succinate <-- Succinate
GPR,b0241 or b0929 or b1377 or b2215
Lower bound,-400
Upper bound,-400


In [40]:
cobra.Reaction("SUCCtex")

Reaction identifier,SUCCtex
Name,
Memory address,0x7f0434934880
Stoichiometry,--> -->
GPR,
Lower bound,0.0
Upper bound,1000.0


In [27]:
O2tex = model.reactions.get_by_id("O2tex")
O2tex

Reaction identifier,O2tex
Name,Oxygen transport via diffusion (extracellular to periplasm)
Memory address,0x7f0880d21d90
Stoichiometry,o2_e <=> o2_p O2 O2 <=> O2 O2
GPR,b0241 or b0929 or b1377 or b2215
Lower bound,-1000.0
Upper bound,1000.0


In [31]:
O2tex.bounds = (0, 100)

In [32]:
O2tex

Reaction identifier,O2tex
Name,Oxygen transport via diffusion (extracellular to periplasm)
Memory address,0x7f0880d21d90
Stoichiometry,o2_e --> o2_p O2 O2 --> O2 O2
GPR,b0241 or b0929 or b1377 or b2215
Lower bound,0
Upper bound,100


In [24]:
model.solver

```matlab
model.c(strcmp(model.rxns, 'BIOMASS_Ec_iJO1366_core_53p95M')) = 1;
model = changeRxnBounds(model, 'EX_glc__D_e', -100, 'l'); 
model = changeRxnBounds(model, 'EX_o2_e', -100, 'l'); 
model = changeRxnBounds(model, 'EX_so4_e', -100, 'l'); 
model = changeRxnBounds(model, 'EX_nh4_e', -100, 'l'); 
model = changeRxnBounds(model, 'EX_cit_e', -100, 'l'); 
model = changeRxnBounds(model, 'EX_glyc_e', -100, 'l'); 
```

Chage objective to `EX_succ_e`

In [45]:
model.objective = 'EX_succ_e'

In [46]:
print(model.objective.expression)
print(model.objective.direction)

1.0*EX_succ_e - 1.0*EX_succ_e_reverse_a9039
max


In [47]:
solution = model.optimize()
print(solution)

<Solution 17.096 at 0x7f087547f7c0>


In [48]:
solution.objective_value

17.096428571428582

In [49]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_e,8.386,1,12.26%
glc__D_e,EX_glc__D_e,10,6,87.74%
o2_e,EX_o2_e,0.1625,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
h2o_e,EX_h2o_e,-8.711,0,0.00%
h_e,EX_h_e,-34.19,0,0.00%
succ_e,EX_succ_e,-17.1,4,100.00%


In [50]:
model.metabolites.succ_e.summary()

Percent,Flux,Reaction,Definition
100.00%,17.1,SUCCtex,succ_e <=> succ_p
Percent,Flux,Reaction,Definition
100.00%,-17.1,EX_succ_e,succ_e -->


In [51]:
model.reactions.SUCCtex.summary()

In [53]:
model.reactions.O2tex.summary()

In [52]:
model.reactions.MALS.summary()

In [61]:
model.reactions.EX_nh4_e.summary()

KeyError: 'EX_nh4_e'

In [59]:
model.reactions.get_by_id("EX_nh4_e")

Reaction identifier,EX_nh4_e
Name,Ammonia exchange
Memory address,0x7f0847ed98b0
Stoichiometry,nh4_e <=> Ammonium <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [60]:
model.reactions.EX_nh4_e.summary() # why there is no summary for EX_nh4_e?

KeyError: 'EX_nh4_e'

Change the bound and then do the FBA simulation?

Change the bound seams not working

how about change the coversion rate?

The `knock_out_model_genes` function will evaluate the GPR and set the upper and lower bounds to 0 if the reaction is knocked out.

In [35]:
with model:
    model.reactions.ACONTa.knock_out()
    print('ACONTa knocked out: ', model.optimize())

ACONTa knocked out:  <Solution infeasible at 0x7f087565cbb0>


/home/chenjunyu/miniconda3/envs/cobra/lib/python3.8/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


In [ ]:
ACONTa